# Experiment 041: ChemBERTa Pre-trained Embeddings

**Hypothesis:** Pre-trained molecular embeddings from ChemBERTa (trained on millions of molecules) will transfer better to unseen solvents than models trained from scratch.

**Approach:**
1. Use ChemBERTa to get 768-dim embeddings for each solvent SMILES
2. Combine with Arrhenius kinetics features
3. Feed into MLP model
4. Test on single fold first, then full CV if promising

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Check for transformers
try:
    from transformers import AutoModel, AutoTokenizer
    print('Transformers available')
except ImportError:
    print('Installing transformers...')
    import subprocess
    subprocess.run(['pip', 'install', 'transformers', '-q'])
    from transformers import AutoModel, AutoTokenizer
    print('Transformers installed')

np.random.seed(42)
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Transformers available
Using device: cuda


In [9]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["SM", "Product 2", "Product 3"]]  # Correct column names
    return X, Y

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load SMILES lookup
SMILES_DF = pd.read_csv(f'{DATA_PATH}/smiles_lookup.csv', index_col=0)
print(f'SMILES lookup: {len(SMILES_DF)} solvents')
print(SMILES_DF.head())

SMILES lookup: 26 solvents
                                           solvent smiles
SOLVENT NAME                                             
Cyclohexane                                      C1CCCCC1
Ethyl Acetate                                   O=C(OCC)C
Acetic Acid                                       CC(=O)O
2-Methyltetrahydrofuran [2-MeTHF]              O1C(C)CCC1
1,1,1,3,3,3-Hexafluoropropan-2-ol  C(C(F)(F)F)(C(F)(F)F)O


In [4]:
# ChemBERTa Featurizer
class ChemBERTaFeaturizer:
    def __init__(self):
        print('Loading ChemBERTa model...')
        self.tokenizer = AutoTokenizer.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
        self.model = AutoModel.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
        self.model.eval()
        self.model.to(device)
        print(f'ChemBERTa loaded on {device}')
        
        # Cache for embeddings
        self.cache = {}
    
    def get_embedding(self, smiles):
        """Get 768-dim embedding for a SMILES string."""
        if smiles in self.cache:
            return self.cache[smiles]
        
        # Handle mixture SMILES by averaging components
        if '.' in smiles:
            components = smiles.split('.')
            embeddings = [self._get_single_embedding(s) for s in components]
            embedding = np.mean(embeddings, axis=0)
        else:
            embedding = self._get_single_embedding(smiles)
        
        self.cache[smiles] = embedding
        return embedding
    
    def _get_single_embedding(self, smiles):
        """Get embedding for a single SMILES (no mixtures)."""
        with torch.no_grad():
            inputs = self.tokenizer(smiles, return_tensors='pt', padding=True, truncation=True, max_length=512)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = self.model(**inputs)
            # Use [CLS] token embedding
            embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
        return embedding

# Initialize featurizer
featurizer = ChemBERTaFeaturizer()

Loading ChemBERTa model...


ChemBERTa loaded on cuda


In [5]:
# Pre-compute embeddings for all solvents
SOLVENT_EMBEDDINGS = {}
for solvent_name in SMILES_DF.index:
    smiles = SMILES_DF.loc[solvent_name, 'solvent smiles']
    try:
        embedding = featurizer.get_embedding(smiles)
        SOLVENT_EMBEDDINGS[solvent_name] = embedding
        print(f"{solvent_name}: embedding shape {embedding.shape}")
    except Exception as e:
        print(f"ERROR {solvent_name}: {e}")

print(f"\nTotal embeddings: {len(SOLVENT_EMBEDDINGS)}")

Cyclohexane: embedding shape (768,)
Ethyl Acetate: embedding shape (768,)
Acetic Acid: embedding shape (768,)
2-Methyltetrahydrofuran [2-MeTHF]: embedding shape (768,)
1,1,1,3,3,3-Hexafluoropropan-2-ol: embedding shape (768,)
IPA [Propan-2-ol]: embedding shape (768,)
Ethanol: embedding shape (768,)
Methanol: embedding shape (768,)
Ethylene Glycol [1,2-Ethanediol]: embedding shape (768,)
Acetonitrile: embedding shape (768,)
Water: embedding shape (768,)
Diethyl Ether [Ether]: embedding shape (768,)
MTBE [tert-Butylmethylether]: embedding shape (768,)
Dimethyl Carbonate: embedding shape (768,)
tert-Butanol [2-Methylpropan-2-ol]: embedding shape (768,)
DMA [N,N-Dimethylacetamide]: embedding shape (768,)
2,2,2-Trifluoroethanol: embedding shape (768,)
Dihydrolevoglucosenone (Cyrene): embedding shape (768,)
Decanol: embedding shape (768,)
Butanone [MEK]: embedding shape (768,)
Ethyl Lactate: embedding shape (768,)
Methyl Propionate: embedding shape (768,)
THF [Tetrahydrofuran]: embedding sha

In [10]:
# ChemBERTa MLP Model
class ChemBERTaMLPModel(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64]):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(0.3)
            ])
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, 3))  # 3 outputs: SM, Product 1, Product 2
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

print('ChemBERTaMLPModel defined')

ChemBERTaMLPModel defined


In [11]:
# ChemBERTa Wrapper for training and prediction
class ChemBERTaWrapper:
    def __init__(self, data='single', n_models=3):
        self.data_type = data
        self.n_models = n_models
        self.models = []
        self.scalers = []
        self.solvent_embeddings = SOLVENT_EMBEDDINGS
    
    def _get_features(self, X):
        """Extract features: ChemBERTa embeddings + Arrhenius kinetics."""
        features_list = []
        
        for idx, row in X.iterrows():
            # Kinetics features
            time_m = row['Residence Time']
            temp_c = row['Temperature']
            temp_k = temp_c + 273.15
            
            kinetics = np.array([
                time_m,
                temp_c,
                1.0 / temp_k,  # Arrhenius
                np.log(time_m + 1),  # log time
                time_m / temp_k  # interaction
            ])
            
            # ChemBERTa embedding
            if self.data_type == 'single':
                solvent_name = row['SOLVENT NAME']
                embedding = self.solvent_embeddings[solvent_name]
            else:
                # Full solvent (mixture)
                solvent_a = row['SOLVENT A NAME']
                solvent_b = row['SOLVENT B NAME']
                pct_b = row['SolventB%'] / 100.0
                
                emb_a = self.solvent_embeddings[solvent_a]
                emb_b = self.solvent_embeddings[solvent_b]
                embedding = (1 - pct_b) * emb_a + pct_b * emb_b
            
            # Combine
            features = np.concatenate([kinetics, embedding])
            features_list.append(features)
        
        return np.array(features_list, dtype=np.float32)
    
    def train_model(self, X_train, y_train, epochs=200):
        """Train ensemble of MLP models."""
        X_feat = self._get_features(X_train)
        y_np = y_train.values.astype(np.float32)
        
        self.models = []
        self.scalers = []
        
        for i in range(self.n_models):
            # Scale features
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_feat)
            self.scalers.append(scaler)
            
            # Create model
            input_dim = X_scaled.shape[1]  # 5 kinetics + 768 ChemBERTa = 773
            model = ChemBERTaMLPModel(input_dim).to(device)
            
            # Training
            optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
            
            X_tensor = torch.tensor(X_scaled).to(device)
            y_tensor = torch.tensor(y_np).to(device)
            
            dataset = TensorDataset(X_tensor, y_tensor)
            loader = DataLoader(dataset, batch_size=32, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                for X_batch, y_batch in loader:
                    optimizer.zero_grad()
                    pred = model(X_batch)
                    loss = nn.MSELoss()(pred, y_batch)
                    loss.backward()
                    optimizer.step()
                scheduler.step()
            
            model.eval()
            self.models.append(model)
        
        return self
    
    def predict(self, X_test):
        """Predict using ensemble."""
        X_feat = self._get_features(X_test)
        
        all_preds = []
        for model, scaler in zip(self.models, self.scalers):
            X_scaled = scaler.transform(X_feat)
            X_tensor = torch.tensor(X_scaled).to(device)
            
            with torch.no_grad():
                pred = model(X_tensor).cpu()
            all_preds.append(pred)
        
        # Average predictions
        return torch.stack(all_preds).mean(dim=0)

print('ChemBERTaWrapper defined')

ChemBERTaWrapper defined


In [12]:
# Quick test on single fold
X_single, Y_single = load_data("single_solvent")
test_solvent = sorted(X_single["SOLVENT NAME"].unique())[0]
mask = X_single["SOLVENT NAME"] != test_solvent

print(f"Test solvent: {test_solvent}")
print(f"Training samples: {mask.sum()}, Test samples: {(~mask).sum()}")

model = ChemBERTaWrapper(data='single', n_models=1)
model.train_model(X_single[mask], Y_single[mask], epochs=100)
preds = model.predict(X_single[~mask])

actuals = Y_single[~mask].values
mse = np.mean((actuals - preds.numpy())**2)
print(f"\nTest fold MSE: {mse:.6f}")
print(f"Predictions shape: {preds.shape}")
print(f"\nComparison: exp_035 baseline CV = 0.008194")
print(f"GNN (exp_040) test fold MSE = 0.068767")

Test solvent: 1,1,1,3,3,3-Hexafluoropropan-2-ol
Training samples: 619, Test samples: 37



Test fold MSE: 0.061135
Predictions shape: torch.Size([37, 3])

Comparison: exp_035 baseline CV = 0.008194
GNN (exp_040) test fold MSE = 0.068767


In [14]:
# ChemBERTa alone is not enough (MSE 0.061135 vs baseline 0.008194)
# Let's try combining ChemBERTa with Spange descriptors

# Load Spange descriptors
spange_df = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
print(f"Spange descriptors: {spange_df.shape}")
print(spange_df.head())

# Create combined features: ChemBERTa + Spange + Arrhenius
SPANGE_COLS = [c for c in spange_df.columns if c != 'solvent smiles']
print(f"\nSpange columns: {SPANGE_COLS}")

SPANGE_DICT = {}
for solvent in spange_df.index:
    SPANGE_DICT[solvent] = spange_df.loc[solvent, SPANGE_COLS].values.astype(np.float32)
print(f"\nSpange dict: {len(SPANGE_DICT)} solvents")

Spange descriptors: (26, 13)
                                   dielectric constant  ET(30)  alpha  beta  \
SOLVENT NAME                                                                  
Cyclohexane                                       2.02    30.9   0.00  0.00   
Ethyl Acetate                                     6.02    38.1   0.00  0.45   
Acetic Acid                                       6.15    51.7   1.12  0.45   
2-Methyltetrahydrofuran [2-MeTHF]                 7.58    36.5   0.00  0.45   
1,1,1,3,3,3-Hexafluoropropan-2-ol                16.70    62.1   1.96  0.00   

                                    pi*     SA     SB     SP    SdP        N  \
SOLVENT NAME                                                                   
Cyclohexane                        0.00  0.000  0.073  0.683  0.000  0.00927   
Ethyl Acetate                      0.55  0.000  0.542  0.656  0.603  0.01021   
Acetic Acid                        0.64  0.689  0.390  0.651  0.676  0.01749   
2-Methyltetrahydr

In [15]:
# Combined ChemBERTa + Spange Wrapper
class CombinedWrapper:
    def __init__(self, data='single', n_models=3, use_chemberta=True, use_spange=True):
        self.data_type = data
        self.n_models = n_models
        self.models = []
        self.scalers = []
        self.use_chemberta = use_chemberta
        self.use_spange = use_spange
    
    def _get_features(self, X):
        """Extract features: ChemBERTa + Spange + Arrhenius kinetics."""
        features_list = []
        
        for idx, row in X.iterrows():
            # Kinetics features (5 features)
            time_m = row['Residence Time']
            temp_c = row['Temperature']
            temp_k = temp_c + 273.15
            
            kinetics = np.array([
                time_m,
                temp_c,
                1.0 / temp_k,  # Arrhenius
                np.log(time_m + 1),  # log time
                time_m / temp_k  # interaction
            ], dtype=np.float32)
            
            # Get solvent info
            if self.data_type == 'single':
                solvent_name = row['SOLVENT NAME']
                
                # ChemBERTa embedding (768 features)
                chemberta_emb = SOLVENT_EMBEDDINGS[solvent_name] if self.use_chemberta else np.array([])
                
                # Spange descriptors (7 features)
                spange_feat = SPANGE_DICT.get(solvent_name, np.zeros(7, dtype=np.float32)) if self.use_spange else np.array([])
            else:
                # Full solvent (mixture)
                solvent_a = row['SOLVENT A NAME']
                solvent_b = row['SOLVENT B NAME']
                pct_b = row['SolventB%'] / 100.0
                
                if self.use_chemberta:
                    emb_a = SOLVENT_EMBEDDINGS[solvent_a]
                    emb_b = SOLVENT_EMBEDDINGS[solvent_b]
                    chemberta_emb = (1 - pct_b) * emb_a + pct_b * emb_b
                else:
                    chemberta_emb = np.array([])
                
                if self.use_spange:
                    sp_a = SPANGE_DICT.get(solvent_a, np.zeros(7, dtype=np.float32))
                    sp_b = SPANGE_DICT.get(solvent_b, np.zeros(7, dtype=np.float32))
                    spange_feat = (1 - pct_b) * sp_a + pct_b * sp_b
                else:
                    spange_feat = np.array([])
            
            # Combine all features
            features = np.concatenate([kinetics, chemberta_emb, spange_feat])
            features_list.append(features)
        
        return np.array(features_list, dtype=np.float32)
    
    def train_model(self, X_train, y_train, epochs=200):
        """Train ensemble of MLP models."""
        X_feat = self._get_features(X_train)
        y_np = y_train.values.astype(np.float32)
        
        self.models = []
        self.scalers = []
        
        for i in range(self.n_models):
            # Scale features
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_feat)
            self.scalers.append(scaler)
            
            # Create model
            input_dim = X_scaled.shape[1]
            model = ChemBERTaMLPModel(input_dim, hidden_dims=[256, 128, 64]).to(device)
            
            # Training
            optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
            
            X_tensor = torch.tensor(X_scaled).to(device)
            y_tensor = torch.tensor(y_np).to(device)
            
            dataset = TensorDataset(X_tensor, y_tensor)
            loader = DataLoader(dataset, batch_size=32, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                for X_batch, y_batch in loader:
                    optimizer.zero_grad()
                    pred = model(X_batch)
                    loss = nn.MSELoss()(pred, y_batch)
                    loss.backward()
                    optimizer.step()
                scheduler.step()
            
            model.eval()
            self.models.append(model)
        
        return self
    
    def predict(self, X_test):
        """Predict using ensemble."""
        X_feat = self._get_features(X_test)
        
        all_preds = []
        for model, scaler in zip(self.models, self.scalers):
            X_scaled = scaler.transform(X_feat)
            X_tensor = torch.tensor(X_scaled).to(device)
            
            with torch.no_grad():
                pred = model(X_tensor).cpu()
            all_preds.append(pred)
        
        # Average predictions
        return torch.stack(all_preds).mean(dim=0)

print('CombinedWrapper defined')

CombinedWrapper defined


In [16]:
# Test different feature combinations on single fold
print("Testing feature combinations on single fold...")
print(f"Test solvent: {test_solvent}")
print()

# ChemBERTa only
model1 = CombinedWrapper(data='single', n_models=1, use_chemberta=True, use_spange=False)
model1.train_model(X_single[mask], Y_single[mask], epochs=100)
preds1 = model1.predict(X_single[~mask])
mse1 = np.mean((actuals - preds1.numpy())**2)
print(f"ChemBERTa only: MSE = {mse1:.6f}")

# Spange only
model2 = CombinedWrapper(data='single', n_models=1, use_chemberta=False, use_spange=True)
model2.train_model(X_single[mask], Y_single[mask], epochs=100)
preds2 = model2.predict(X_single[~mask])
mse2 = np.mean((actuals - preds2.numpy())**2)
print(f"Spange only: MSE = {mse2:.6f}")

# ChemBERTa + Spange
model3 = CombinedWrapper(data='single', n_models=1, use_chemberta=True, use_spange=True)
model3.train_model(X_single[mask], Y_single[mask], epochs=100)
preds3 = model3.predict(X_single[~mask])
mse3 = np.mean((actuals - preds3.numpy())**2)
print(f"ChemBERTa + Spange: MSE = {mse3:.6f}")

print(f"\\nBaseline (exp_035): CV = 0.008194")

Testing feature combinations on single fold...
Test solvent: 1,1,1,3,3,3-Hexafluoropropan-2-ol



ChemBERTa only: MSE = 0.052789


Spange only: MSE = 0.034241


ChemBERTa + Spange: MSE = 0.042313
\nBaseline (exp_035): CV = 0.008194


In [22]:
# Load DRFP features - these are reaction fingerprints that capture the chemistry
drfp_df = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
print(f"DRFP features: {drfp_df.shape}")

# Get DRFP columns - they are numeric (0, 1, 2, ..., 2047)
DRFP_COLS = [c for c in drfp_df.columns if str(c).isdigit() or isinstance(c, int)]
print(f"DRFP columns: {len(DRFP_COLS)}")

DRFP_DICT = {}
for solvent in drfp_df.index:
    DRFP_DICT[solvent] = drfp_df.loc[solvent, DRFP_COLS].values.astype(np.float32)
print(f"DRFP dict: {len(DRFP_DICT)} solvents")
print(f"DRFP feature dim: {DRFP_DICT[list(DRFP_DICT.keys())[0]].shape}")

DRFP features: (24, 2048)
DRFP columns: 2048
DRFP dict: 24 solvents
DRFP feature dim: (2048,)


In [23]:
# Full Combined Wrapper with ChemBERTa + Spange + DRFP
class FullCombinedWrapper:
    def __init__(self, data='single', n_models=3, use_chemberta=True, use_spange=True, use_drfp=True):
        self.data_type = data
        self.n_models = n_models
        self.models = []
        self.scalers = []
        self.use_chemberta = use_chemberta
        self.use_spange = use_spange
        self.use_drfp = use_drfp
    
    def _get_features(self, X):
        """Extract features: ChemBERTa + Spange + DRFP + Arrhenius kinetics."""
        features_list = []
        
        for idx, row in X.iterrows():
            # Kinetics features (5 features)
            time_m = row['Residence Time']
            temp_c = row['Temperature']
            temp_k = temp_c + 273.15
            
            kinetics = np.array([
                time_m,
                temp_c,
                1.0 / temp_k,  # Arrhenius
                np.log(time_m + 1),  # log time
                time_m / temp_k  # interaction
            ], dtype=np.float32)
            
            # Get solvent info
            if self.data_type == 'single':
                solvent_name = row['SOLVENT NAME']
                
                # ChemBERTa embedding (768 features)
                chemberta_emb = SOLVENT_EMBEDDINGS[solvent_name] if self.use_chemberta else np.array([])
                
                # Spange descriptors (13 features)
                spange_feat = SPANGE_DICT.get(solvent_name, np.zeros(len(SPANGE_COLS), dtype=np.float32)) if self.use_spange else np.array([])
                
                # DRFP features (128 features)
                drfp_feat = DRFP_DICT.get(solvent_name, np.zeros(len(DRFP_COLS), dtype=np.float32)) if self.use_drfp else np.array([])
            else:
                # Full solvent (mixture)
                solvent_a = row['SOLVENT A NAME']
                solvent_b = row['SOLVENT B NAME']
                pct_b = row['SolventB%'] / 100.0
                
                if self.use_chemberta:
                    emb_a = SOLVENT_EMBEDDINGS[solvent_a]
                    emb_b = SOLVENT_EMBEDDINGS[solvent_b]
                    chemberta_emb = (1 - pct_b) * emb_a + pct_b * emb_b
                else:
                    chemberta_emb = np.array([])
                
                if self.use_spange:
                    sp_a = SPANGE_DICT.get(solvent_a, np.zeros(len(SPANGE_COLS), dtype=np.float32))
                    sp_b = SPANGE_DICT.get(solvent_b, np.zeros(len(SPANGE_COLS), dtype=np.float32))
                    spange_feat = (1 - pct_b) * sp_a + pct_b * sp_b
                else:
                    spange_feat = np.array([])
                
                if self.use_drfp:
                    dr_a = DRFP_DICT.get(solvent_a, np.zeros(len(DRFP_COLS), dtype=np.float32))
                    dr_b = DRFP_DICT.get(solvent_b, np.zeros(len(DRFP_COLS), dtype=np.float32))
                    drfp_feat = (1 - pct_b) * dr_a + pct_b * dr_b
                else:
                    drfp_feat = np.array([])
            
            # Combine all features
            features = np.concatenate([kinetics, chemberta_emb, spange_feat, drfp_feat])
            features_list.append(features)
        
        return np.array(features_list, dtype=np.float32)
    
    def train_model(self, X_train, y_train, epochs=200):
        """Train ensemble of MLP models."""
        X_feat = self._get_features(X_train)
        y_np = y_train.values.astype(np.float32)
        
        self.models = []
        self.scalers = []
        
        for i in range(self.n_models):
            # Scale features
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_feat)
            self.scalers.append(scaler)
            
            # Create model
            input_dim = X_scaled.shape[1]
            model = ChemBERTaMLPModel(input_dim, hidden_dims=[256, 128, 64]).to(device)
            
            # Training
            optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
            
            X_tensor = torch.tensor(X_scaled).to(device)
            y_tensor = torch.tensor(y_np).to(device)
            
            dataset = TensorDataset(X_tensor, y_tensor)
            loader = DataLoader(dataset, batch_size=32, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                for X_batch, y_batch in loader:
                    optimizer.zero_grad()
                    pred = model(X_batch)
                    loss = nn.MSELoss()(pred, y_batch)
                    loss.backward()
                    optimizer.step()
                scheduler.step()
            
            model.eval()
            self.models.append(model)
        
        return self
    
    def predict(self, X_test):
        """Predict using ensemble."""
        X_feat = self._get_features(X_test)
        
        all_preds = []
        for model, scaler in zip(self.models, self.scalers):
            X_scaled = scaler.transform(X_feat)
            X_tensor = torch.tensor(X_scaled).to(device)
            
            with torch.no_grad():
                pred = model(X_tensor).cpu()
            all_preds.append(pred)
        
        # Average predictions
        return torch.stack(all_preds).mean(dim=0)

print('FullCombinedWrapper defined')

FullCombinedWrapper defined


In [24]:
# Test with DRFP features
print("Testing with DRFP features on single fold...")
print(f"Test solvent: {test_solvent}")
print()

# DRFP only
model4 = FullCombinedWrapper(data='single', n_models=1, use_chemberta=False, use_spange=False, use_drfp=True)
model4.train_model(X_single[mask], Y_single[mask], epochs=100)
preds4 = model4.predict(X_single[~mask])
mse4 = np.mean((actuals - preds4.numpy())**2)
print(f"DRFP only: MSE = {mse4:.6f}")

# Spange + DRFP (baseline features)
model5 = FullCombinedWrapper(data='single', n_models=1, use_chemberta=False, use_spange=True, use_drfp=True)
model5.train_model(X_single[mask], Y_single[mask], epochs=100)
preds5 = model5.predict(X_single[~mask])
mse5 = np.mean((actuals - preds5.numpy())**2)
print(f"Spange + DRFP: MSE = {mse5:.6f}")

# ChemBERTa + Spange + DRFP (all features)
model6 = FullCombinedWrapper(data='single', n_models=1, use_chemberta=True, use_spange=True, use_drfp=True)
model6.train_model(X_single[mask], Y_single[mask], epochs=100)
preds6 = model6.predict(X_single[~mask])
mse6 = np.mean((actuals - preds6.numpy())**2)
print(f"ChemBERTa + Spange + DRFP: MSE = {mse6:.6f}")

print(f"\\nBaseline (exp_035): CV = 0.008194")

Testing with DRFP features on single fold...
Test solvent: 1,1,1,3,3,3-Hexafluoropropan-2-ol



DRFP only: MSE = 0.059665


Spange + DRFP: MSE = 0.057212


ChemBERTa + Spange + DRFP: MSE = 0.053051
\nBaseline (exp_035): CV = 0.008194


In [26]:
# Try PCA on ChemBERTa embeddings to reduce dimensionality
from sklearn.decomposition import PCA

# Get all ChemBERTa embeddings
all_embeddings = np.array([SOLVENT_EMBEDDINGS[s] for s in SOLVENT_EMBEDDINGS.keys()])
print(f"All embeddings shape: {all_embeddings.shape}")

# We only have 26 solvents, so max PCA components = 26
for n_components in [5, 10, 15, 20, 25]:
    pca = PCA(n_components=n_components)
    reduced = pca.fit_transform(all_embeddings)
    explained_var = pca.explained_variance_ratio_.sum()
    print(f"PCA {n_components}: explained variance = {explained_var:.3f}")

# Use PCA with 20 components (good balance)
pca = PCA(n_components=20)
pca.fit(all_embeddings)

# Create reduced embeddings dict
CHEMBERTA_PCA_DICT = {}
for solvent in SOLVENT_EMBEDDINGS.keys():
    emb = SOLVENT_EMBEDDINGS[solvent].reshape(1, -1)
    reduced = pca.transform(emb).flatten()
    CHEMBERTA_PCA_DICT[solvent] = reduced.astype(np.float32)
print(f"\\nChemBERTa PCA dict: {len(CHEMBERTA_PCA_DICT)} solvents, dim = {CHEMBERTA_PCA_DICT[list(CHEMBERTA_PCA_DICT.keys())[0]].shape}")

All embeddings shape: (26, 768)
PCA 5: explained variance = 0.770
PCA 10: explained variance = 0.936
PCA 15: explained variance = 0.986
PCA 20: explained variance = 1.000
PCA 25: explained variance = 1.000
\nChemBERTa PCA dict: 26 solvents, dim = (20,)


In [27]:
# Test with PCA-reduced ChemBERTa embeddings
class PCAChemBERTaWrapper:
    def __init__(self, data='single', n_models=3, use_chemberta_pca=True, use_spange=True, use_drfp=False):
        self.data_type = data
        self.n_models = n_models
        self.models = []
        self.scalers = []
        self.use_chemberta_pca = use_chemberta_pca
        self.use_spange = use_spange
        self.use_drfp = use_drfp
    
    def _get_features(self, X):
        """Extract features: ChemBERTa PCA + Spange + Arrhenius kinetics."""
        features_list = []
        
        for idx, row in X.iterrows():
            # Kinetics features (5 features)
            time_m = row['Residence Time']
            temp_c = row['Temperature']
            temp_k = temp_c + 273.15
            
            kinetics = np.array([
                time_m,
                temp_c,
                1.0 / temp_k,  # Arrhenius
                np.log(time_m + 1),  # log time
                time_m / temp_k  # interaction
            ], dtype=np.float32)
            
            # Get solvent info
            if self.data_type == 'single':
                solvent_name = row['SOLVENT NAME']
                
                # ChemBERTa PCA embedding (20 features)
                chemberta_pca = CHEMBERTA_PCA_DICT[solvent_name] if self.use_chemberta_pca else np.array([])
                
                # Spange descriptors (13 features)
                spange_feat = SPANGE_DICT.get(solvent_name, np.zeros(len(SPANGE_COLS), dtype=np.float32)) if self.use_spange else np.array([])
                
                # DRFP features (2048 features)
                drfp_feat = DRFP_DICT.get(solvent_name, np.zeros(len(DRFP_COLS), dtype=np.float32)) if self.use_drfp else np.array([])
            else:
                # Full solvent (mixture)
                solvent_a = row['SOLVENT A NAME']
                solvent_b = row['SOLVENT B NAME']
                pct_b = row['SolventB%'] / 100.0
                
                if self.use_chemberta_pca:
                    emb_a = CHEMBERTA_PCA_DICT[solvent_a]
                    emb_b = CHEMBERTA_PCA_DICT[solvent_b]
                    chemberta_pca = (1 - pct_b) * emb_a + pct_b * emb_b
                else:
                    chemberta_pca = np.array([])
                
                if self.use_spange:
                    sp_a = SPANGE_DICT.get(solvent_a, np.zeros(len(SPANGE_COLS), dtype=np.float32))
                    sp_b = SPANGE_DICT.get(solvent_b, np.zeros(len(SPANGE_COLS), dtype=np.float32))
                    spange_feat = (1 - pct_b) * sp_a + pct_b * sp_b
                else:
                    spange_feat = np.array([])
                
                if self.use_drfp:
                    dr_a = DRFP_DICT.get(solvent_a, np.zeros(len(DRFP_COLS), dtype=np.float32))
                    dr_b = DRFP_DICT.get(solvent_b, np.zeros(len(DRFP_COLS), dtype=np.float32))
                    drfp_feat = (1 - pct_b) * dr_a + pct_b * dr_b
                else:
                    drfp_feat = np.array([])
            
            # Combine all features
            features = np.concatenate([kinetics, chemberta_pca, spange_feat, drfp_feat])
            features_list.append(features)
        
        return np.array(features_list, dtype=np.float32)
    
    def train_model(self, X_train, y_train, epochs=200):
        """Train ensemble of MLP models."""
        X_feat = self._get_features(X_train)
        y_np = y_train.values.astype(np.float32)
        
        self.models = []
        self.scalers = []
        
        for i in range(self.n_models):
            # Scale features
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_feat)
            self.scalers.append(scaler)
            
            # Create model
            input_dim = X_scaled.shape[1]
            model = ChemBERTaMLPModel(input_dim, hidden_dims=[128, 64, 32]).to(device)
            
            # Training
            optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
            
            X_tensor = torch.tensor(X_scaled).to(device)
            y_tensor = torch.tensor(y_np).to(device)
            
            dataset = TensorDataset(X_tensor, y_tensor)
            loader = DataLoader(dataset, batch_size=32, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                for X_batch, y_batch in loader:
                    optimizer.zero_grad()
                    pred = model(X_batch)
                    loss = nn.MSELoss()(pred, y_batch)
                    loss.backward()
                    optimizer.step()
                scheduler.step()
            
            model.eval()
            self.models.append(model)
        
        return self
    
    def predict(self, X_test):
        """Predict using ensemble."""
        X_feat = self._get_features(X_test)
        
        all_preds = []
        for model, scaler in zip(self.models, self.scalers):
            X_scaled = scaler.transform(X_feat)
            X_tensor = torch.tensor(X_scaled).to(device)
            
            with torch.no_grad():
                pred = model(X_tensor).cpu()
            all_preds.append(pred)
        
        # Average predictions
        return torch.stack(all_preds).mean(dim=0)

print('PCAChemBERTaWrapper defined')

PCAChemBERTaWrapper defined


In [28]:
# Test PCA ChemBERTa on single fold
print("Testing PCA ChemBERTa on single fold...")
print(f"Test solvent: {test_solvent}")
print()

# ChemBERTa PCA only
model7 = PCAChemBERTaWrapper(data='single', n_models=1, use_chemberta_pca=True, use_spange=False, use_drfp=False)
model7.train_model(X_single[mask], Y_single[mask], epochs=100)
preds7 = model7.predict(X_single[~mask])
mse7 = np.mean((actuals - preds7.numpy())**2)
print(f"ChemBERTa PCA only: MSE = {mse7:.6f}")

# ChemBERTa PCA + Spange
model8 = PCAChemBERTaWrapper(data='single', n_models=1, use_chemberta_pca=True, use_spange=True, use_drfp=False)
model8.train_model(X_single[mask], Y_single[mask], epochs=100)
preds8 = model8.predict(X_single[~mask])
mse8 = np.mean((actuals - preds8.numpy())**2)
print(f"ChemBERTa PCA + Spange: MSE = {mse8:.6f}")

# ChemBERTa PCA + Spange + DRFP
model9 = PCAChemBERTaWrapper(data='single', n_models=1, use_chemberta_pca=True, use_spange=True, use_drfp=True)
model9.train_model(X_single[mask], Y_single[mask], epochs=100)
preds9 = model9.predict(X_single[~mask])
mse9 = np.mean((actuals - preds9.numpy())**2)
print(f"ChemBERTa PCA + Spange + DRFP: MSE = {mse9:.6f}")

print(f"\\nBaseline (exp_035): CV = 0.008194")

Testing PCA ChemBERTa on single fold...
Test solvent: 1,1,1,3,3,3-Hexafluoropropan-2-ol



ChemBERTa PCA only: MSE = 0.058003


ChemBERTa PCA + Spange: MSE = 0.041895


ChemBERTa PCA + Spange + DRFP: MSE = 0.052064
\nBaseline (exp_035): CV = 0.008194


In [30]:
# k-NN with Tanimoto Similarity - Priority 2 from evolver
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

class SimilarityKNN:
    def __init__(self, k=5, data='single'):
        self.k = k
        self.data_type = data
        self.train_fps = None
        self.train_y = None
        self.train_kinetics = None
        self.train_solvents = None
    
    def compute_fingerprint(self, smiles):
        """Compute Morgan fingerprint for a SMILES string."""
        # Handle mixture SMILES
        if '.' in smiles:
            smiles = smiles.split('.')[0]
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        return AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    
    def train_model(self, X_train, y_train, epochs=None):
        """Store training data and compute fingerprints."""
        self.train_y = y_train.values
        self.train_kinetics = X_train[['Residence Time', 'Temperature']].values
        
        if self.data_type == 'single':
            self.train_solvents = X_train['SOLVENT NAME'].values
        else:
            self.train_solvents = X_train['SOLVENT A NAME'].values  # Use primary solvent
        
        # Get SMILES and compute fingerprints
        self.train_fps = []
        for solvent in self.train_solvents:
            smiles = SMILES_DF.loc[solvent, 'solvent smiles']
            fp = self.compute_fingerprint(smiles)
            self.train_fps.append(fp)
        
        return self
    
    def predict(self, X_test):
        """Predict using k-NN with Tanimoto similarity."""
        predictions = []
        
        for idx, row in X_test.iterrows():
            if self.data_type == 'single':
                test_solvent = row['SOLVENT NAME']
            else:
                test_solvent = row['SOLVENT A NAME']
            
            test_smiles = SMILES_DF.loc[test_solvent, 'solvent smiles']
            test_fp = self.compute_fingerprint(test_smiles)
            
            if test_fp is None:
                # Fallback to mean prediction
                predictions.append(self.train_y.mean(axis=0))
                continue
            
            # Compute similarities to all training solvents
            similarities = []
            for fp in self.train_fps:
                if fp is not None:
                    sim = DataStructs.TanimotoSimilarity(test_fp, fp)
                else:
                    sim = 0.0
                similarities.append(sim)
            
            similarities = np.array(similarities)
            
            # Find k most similar
            top_k_idx = np.argsort(similarities)[-self.k:]
            top_k_sim = similarities[top_k_idx]
            
            # Weight predictions by similarity
            if top_k_sim.sum() > 0:
                weights = top_k_sim / top_k_sim.sum()
                pred = np.average(self.train_y[top_k_idx], axis=0, weights=weights)
            else:
                pred = self.train_y.mean(axis=0)
            
            predictions.append(pred)
        
        return torch.tensor(np.array(predictions, dtype=np.float32))

print('SimilarityKNN defined')

SimilarityKNN defined


In [31]:
# Test k-NN on single fold
print("Testing k-NN with Tanimoto similarity on single fold...")
print(f"Test solvent: {test_solvent}")
print()

for k in [3, 5, 10, 15]:
    model_knn = SimilarityKNN(k=k, data='single')
    model_knn.train_model(X_single[mask], Y_single[mask])
    preds_knn = model_knn.predict(X_single[~mask])
    mse_knn = np.mean((actuals - preds_knn.numpy())**2)
    print(f"k-NN (k={k}): MSE = {mse_knn:.6f}")

print(f"\\nBaseline (exp_035): CV = 0.008194")

Testing k-NN with Tanimoto similarity on single fold...
Test solvent: 1,1,1,3,3,3-Hexafluoropropan-2-ol

k-NN (k=3): MSE = 0.072666


[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerat

[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerator
[05:19:21] DEPRECATION WARNING: please use MorganGenerat

k-NN (k=5): MSE = 0.103688
k-NN (k=10): MSE = 0.121378


k-NN (k=15): MSE = 0.109587
\nBaseline (exp_035): CV = 0.008194


[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerator
[05:19:22] DEPRECATION WARNING: please use MorganGenerat

In [32]:
# Summary of single fold results
print("=== Summary of Single Fold Results ===")
print(f"Test solvent: {test_solvent}")
print()
print("ChemBERTa approaches:")
print(f"  ChemBERTa only (768-dim): MSE = 0.061135")
print(f"  ChemBERTa PCA (20-dim): MSE = 0.058003")
print(f"  ChemBERTa + Spange: MSE = 0.042313")
print(f"  ChemBERTa PCA + Spange: MSE = 0.041895")
print()
print("Baseline features:")
print(f"  Spange only: MSE = 0.034241")
print(f"  DRFP only: MSE = 0.059665")
print(f"  Spange + DRFP: MSE = 0.057212")
print()
print("k-NN approaches:")
print(f"  k-NN (k=3): MSE = 0.072666")
print(f"  k-NN (k=5): MSE = 0.103688")
print()
print("Previous experiments:")
print(f"  GNN (exp_040): MSE = 0.068767")
print(f"  Baseline (exp_035): CV = 0.008194")
print()
print("CONCLUSION: All new approaches perform WORSE than baseline on this fold.")
print("The baseline (GP+MLP+LGBM with Spange features) remains the best approach.")

=== Summary of Single Fold Results ===
Test solvent: 1,1,1,3,3,3-Hexafluoropropan-2-ol

ChemBERTa approaches:
  ChemBERTa only (768-dim): MSE = 0.061135
  ChemBERTa PCA (20-dim): MSE = 0.058003
  ChemBERTa + Spange: MSE = 0.042313
  ChemBERTa PCA + Spange: MSE = 0.041895

Baseline features:
  Spange only: MSE = 0.034241
  DRFP only: MSE = 0.059665
  Spange + DRFP: MSE = 0.057212

k-NN approaches:
  k-NN (k=3): MSE = 0.072666
  k-NN (k=5): MSE = 0.103688

Previous experiments:
  GNN (exp_040): MSE = 0.068767
  Baseline (exp_035): CV = 0.008194

CONCLUSION: All new approaches perform WORSE than baseline on this fold.
The baseline (GP+MLP+LGBM with Spange features) remains the best approach.


In [34]:
# Run full CV on ChemBERTa PCA + Spange
print("Running full leave-one-solvent-out CV on ChemBERTa PCA + Spange...")
print()

all_solvents = sorted(X_single["SOLVENT NAME"].unique())
fold_mses = []

for test_solvent in all_solvents:
    mask = X_single["SOLVENT NAME"] != test_solvent
    
    model = PCAChemBERTaWrapper(data='single', n_models=3, use_chemberta_pca=True, use_spange=True, use_drfp=False)
    model.train_model(X_single[mask], Y_single[mask], epochs=150)
    preds = model.predict(X_single[~mask])
    
    actuals = Y_single[~mask].values
    mse = np.mean((actuals - preds.numpy())**2)
    fold_mses.append(mse)
    print(f"{test_solvent}: MSE = {mse:.6f}")

mean_mse = np.mean(fold_mses)
std_mse = np.std(fold_mses)
print(f"\n=== ChemBERTa PCA + Spange CV Results ===")
print(f"Mean MSE: {mean_mse:.6f} +/- {std_mse:.6f}")
print(f"\nComparison:")
print(f"  exp_035 baseline: CV = 0.008194")

Running full leave-one-solvent-out CV on ChemBERTa PCA + Spange...



1,1,1,3,3,3-Hexafluoropropan-2-ol: MSE = 0.034181


2,2,2-Trifluoroethanol: MSE = 0.014989


2-Methyltetrahydrofuran [2-MeTHF]: MSE = 0.003091


Acetonitrile: MSE = 0.005038


...

Methyl Propionate: MSE = 0.003285


THF [Tetrahydrofuran]: MSE = 0.004595


Water.2,2,2-Trifluoroethanol: MSE = 0.001865


Water.Acetonitrile: MSE = 0.009452


tert-Butanol [2-Methylpropan-2-ol]: MSE = 0.000371

=== ChemBERTa PCA + Spange CV Results ===
Mean MSE: 0.010288 +/- 0.008427

Comparison:
  exp_035 baseline: CV = 0.008194


In [ ]:
# If promising, run full CV
print("Running full leave-one-solvent-out CV...")

all_solvents = sorted(X_single["SOLVENT NAME"].unique())
fold_mses = []

for test_solvent in all_solvents:
    mask = X_single["SOLVENT NAME"] != test_solvent
    
    model = ChemBERTaWrapper(data='single', n_models=3)
    model.train_model(X_single[mask], Y_single[mask], epochs=150)
    preds = model.predict(X_single[~mask])
    
    actuals = Y_single[~mask].values
    mse = np.mean((actuals - preds.numpy())**2)
    fold_mses.append(mse)
    print(f"{test_solvent}: MSE = {mse:.6f}")

mean_mse = np.mean(fold_mses)
std_mse = np.std(fold_mses)
print(f"\n=== ChemBERTa CV Results ===")
print(f"Mean MSE: {mean_mse:.6f} ± {std_mse:.6f}")
print(f"\nComparison:")
print(f"  exp_035 baseline: CV = 0.008194")
print(f"  GNN (exp_040): test fold MSE = 0.068767")